In [ ]:
# !pip uninstall -y transformers
# !pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip uninstall -y transformers
!pip install -U transformers

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 99.1 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_hf_image_prompt_captioned_fullset.csv to test_hf_image_prompt_captioned_fullset.csv
Saving train_hf_image_prompt_captioned_fullset.csv to train_hf_image_prompt_captioned_fullset.csv
Saving val_hf_image_prompt_captioned_fullset.csv to val_hf_image_prompt_captioned_fullset.csv


In [ ]:
import pandas as pd
from datasets import Dataset

df_train = pd.read_csv("train_hf_image_prompt_captioned_fullset.csv")
df_val = pd.read_csv("val_hf_image_prompt_captioned_fullset.csv")

dataset = {
    "train": Dataset.from_pandas(df_train),
    "validation": Dataset.from_pandas(df_val)
}

In [ ]:
def format_prompt_only(example):
    return {"text": example["prompt"]}

dataset["train"] = dataset["train"].map(format_prompt_only).remove_columns(
    [col for col in dataset["train"].column_names if col != "text"]
)

dataset["validation"] = dataset["validation"].map(format_prompt_only).remove_columns(
    [col for col in dataset["validation"].column_names if col != "text"]
)

Map:   0%|          | 0/9470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1184 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = {
    "train": dataset["train"].map(tokenize, batched=True),
    "validation": dataset["validation"].map(tokenize, batched=True)
}

Map:   0%|          | 0/9470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1184 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
!ls

drive
sample_data
test_hf_image_prompt_captioned_fullset.csv
train_hf_image_prompt_captioned_fullset.csv
val_hf_image_prompt_captioned_fullset.csv


In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    gradient_checkpointing=True,
    greater_is_better=False,
    lr_scheduler_type="cosine",
    optim="paged_adamw_32bit",
    warmup_steps=200,
    save_total_limit=2,
    fp16=False,
    bf16=True,
    logging_steps=100,
    report_to=["none"],
)


In [ ]:
from transformers import EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)


<ipython-input-15-2c54e092fb7d>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
print("Training finished.")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,2.694400,2.469002
2,1.919300,2.081762
3,1.172200,2.042158
4,0.723300,2.188228
5,0.523400,2.324729
6,0.398300,2.374582
7,0.334800,2.414263


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Training finished.


In [ ]:
model.save_pretrained("/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly")
tokenizer.save_pretrained("/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly")

('/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly/tokenizer_config.json',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly/special_tokens_map.json',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly/chat_template.jinja',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly/vocab.json',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly/merges.txt',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly/added_tokens.json',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly/tokenizer.json')

In [ ]:
!ls -lh "/content/drive/MyDrive/[NLP] Project/qwen2b-promptonly"

total 5.8G
-rw------- 1 root root   80 Jun 10 10:43 added_tokens.json
-rw------- 1 root root  328 Jun 10 10:43 chat_template.jinja
drwx------ 2 root root 4.0K Jun 10 10:42 checkpoint-2072
drwx------ 2 root root 4.0K Jun 10 09:35 checkpoint-888
-rw------- 1 root root  683 Jun 10 10:43 config.json
-rw------- 1 root root  242 Jun 10 10:43 generation_config.json
-rw------- 1 root root 1.6M Jun 10 10:43 merges.txt
-rw------- 1 root root 4.7G Jun 10 10:43 model-00001-of-00002.safetensors
-rw------- 1 root root 1.1G Jun 10 10:43 model-00002-of-00002.safetensors
-rw------- 1 root root  28K Jun 10 10:43 model.safetensors.index.json
-rw------- 1 root root  367 Jun 10 10:43 special_tokens_map.json
-rw------- 1 root root  973 Jun 10 10:43 tokenizer_config.json
-rw------- 1 root root  11M Jun 10 10:43 tokenizer.json
-rw------- 1 root root 2.7M Jun 10 10:43 vocab.json


In [ ]:
model.save_pretrained("/content/drive/MyDrive/[NLP] Project/qwen2b-final", max_shard_size="10GB", safe_serialization=True)
tokenizer.save_pretrained("/content/drive/MyDrive/[NLP] Project/qwen2b-final")

('/content/drive/MyDrive/[NLP] Project/qwen2b-final/tokenizer_config.json',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-final/special_tokens_map.json',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-final/chat_template.jinja',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-final/vocab.json',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-final/merges.txt',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-final/added_tokens.json',
 '/content/drive/MyDrive/[NLP] Project/qwen2b-final/tokenizer.json')

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
df_test = pd.read_csv("test_hf_image_prompt_captioned_fullset.csv")
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False
)

Device set to use cuda:0


In [ ]:
input_texts = df_test["prompt"].tolist()

reference_texts = df_test["prompt"].tolist()

In [ ]:
from tqdm import tqdm

predictions = [
    generator(text, max_length=64)[0]['generated_text']
    for text in tqdm(input_texts)
]

  0%|          | 0/1184 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  1%|          | 10/1184 [01:49<3:33:29, 10.91s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=256) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://

In [ ]:
import pandas as pd

df_eval = pd.DataFrame({
    "input_prompts": input_texts,
    "reference_prompt": reference_texts,
    "generated_prompt": predictions
})

df_eval.to_csv("QWEN_test_predictions.csv", index=False)

In [ ]:
from google.colab import files

files.download("QWEN_test_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
import evaluate

# BLEU
bleu = evaluate.load("bleu")
bleu_result = bleu.compute(predictions=predictions, references=[[ref] for ref in reference_texts])
print("BLEU score:", bleu_result["bleu"])

# BERTScore
bertscore = evaluate.load("bertscore")
bertscore_result = bertscore.compute(predictions=predictions, references=reference_texts, lang="en")
print("BERTScore F1:", sum(bertscore_result["f1"]) / len(bertscore_result["f1"]))

BLEU score: 0.025639717664933497


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.7842572448221413
